In [10]:
#1
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
import matplotlib.pyplot as plt
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.functions import StepFunction
import pickle


In [11]:
#2
# Load data
data = pd.read_csv("data/Recurrence free survival in breast cancer patients.csv")

# Define continuous and categorical variables
CONTINUOUS = ["age", "size", "nodes", "prog", "oest"]
CATEGORICAL = ["treat", "men", "grade"]

# One-hot encode categorical vars
data = pd.get_dummies(data, columns=CATEGORICAL, drop_first=True)

# Scale continuous features
# scaler = StandardScaler()
# data[CONTINUOUS] = scaler.fit_transform(data[CONTINUOUS])


In [12]:
DatTesting

,id,age,size,nodes,prog,oest,time,status,treat_1,men_2,grade_2,grade_3
611,449,48,22,4,14,0,563,1,False,True,True,False
612,448,52,35,1,8,5,308,1,False,True,False,True
613,447,60,21,1,58,701,687,1,False,True,True,False
614,443,48,35,10,2,222,455,1,False,False,False,True
615,441,59,27,20,9,2,624,1,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
681,288,68,14,6,40,68,573,1,True,True,True,False
682,280,64,24,2,41,80,1601,1,False,True,True,False
683,276,58,56,11,51,50,956,1,True,True,False,False
684,330,51,34,3,13,12,1918,1,True,False,True,False


In [13]:
#3
# Reset row order
data = data.sort_values("status").reset_index(drop=True)

# Select last 100 uncensored rows for test set
uncensored = data[data["status"] == 1]
DatTesting = uncensored.iloc[-75:]
DatTraining = data.drop(DatTesting.index)  # include both censored and uncensored

# Define predictors
PREDS = ["age", "size", "nodes", "prog", "oest", "treat_1", "men_2", "grade_2", "grade_3"]
X_train = DatTraining[PREDS].values
X_test = DatTesting[PREDS].values

# Format targets as structured arrays
y_train = np.array([(bool(e), float(t)) for e, t in zip(DatTraining["status"], DatTraining["time"])],
                   dtype=[("event", "bool"), ("time", "float64")])
y_test = np.array([(bool(e), float(t)) for e, t in zip(DatTesting["status"], DatTesting["time"])],
                  dtype=[("event", "bool"), ("time", "float64")])


In [14]:
#4
# Fit Cox Proportional Hazards model
coxph = CoxPHSurvivalAnalysis()
coxph.fit(X_train, y_train)

# Save the trained model to file
with open("coxph_model.pkl", "wb") as f:
    pickle.dump(coxph, f)


In [15]:
def find_nearest_arg(array, value):
    # Find the index of the nearest element of the array to the given value
    idx = (np.abs(array - value)).argmin()
    return idx

Survivors = coxph.predict_survival_function(X_test, return_array=True)

MedianTimeTraining = []

for s in Survivors:
    MedianTimeTraining.append(coxph.unique_times_[find_nearest_arg(s, 0.5)])

#DatTraining["PredictedTime"] = MedianTimeTraining

In [16]:
root_mean_squared_error(y_test["time"],MedianTimeTraining)

1162.540602875157

In [17]:
#5
# Predict survival functions
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.functions import StepFunction

# Predict median survival from risk scores
def predict_median_survival(model, X, y):
    risk_scores = model.predict(X)
    sorted_idx = np.argsort(-risk_scores)
    durations = y["time"][sorted_idx]
    events = y["event"][sorted_idx]

    # Generate step function from sorted durations
    km_times, km_surv = kaplan_meier_estimator(events, durations)
    sf = StepFunction(km_times, km_surv)

    # Predict medians using step function
    medians = []
    for score in model.predict(X):
        # Assumption: higher risk = shorter survival
        s = StepFunction(km_times, np.power(km_surv, np.exp(score)))
        below_half = s(sf.x) < 0.5
        if np.any(below_half):
            median_idx = np.argmax(below_half)
            medians.append(sf.x[median_idx])
        else:
            medians.append(np.nan)
    return np.array(medians)

# --- Test set ---
c_index_test = concordance_index_censored(y_test["event"], y_test["time"], coxph.predict(X_test))[0]
print(f"C-index (Test): {c_index_test:.4f}")

med_test = predict_median_survival(coxph, X_test, y_test)
valid_test = ~np.isnan(med_test)


C-index (Test): 0.5942


In [ ]:
model = coxph
X = X_test
y = y_test

risk_scores = model.predict(X)
sorted_idx = np.argsort(-risk_scores)
durations = y["time"][sorted_idx]
events = y["event"][sorted_idx]

# Generate step function from sorted durations
km_times, km_surv = kaplan_meier_estimator(events, durations)
sf = StepFunction(km_times, km_surv)



In [ ]:
sf

In [ ]:
# Predict medians using step function
medians = []
for score in model.predict(X):
    # Assumption: higher risk = shorter survival
    s = StepFunction(km_times, np.power(km_surv, np.exp(score)))
    below_half = s(sf.x) < 0.5
    if np.any(below_half):
        median_idx = np.argmax(below_half)
        medians.append(sf.x[median_idx])
    else:
        medians.append(np.nan)

In [ ]:
med_test

In [ ]:
root_mean_squared_error(y_test["time"], med_test)

In [ ]:
med_test[valid_test]

In [ ]:
if valid_test.sum() > 0:
    rmse_test = root_mean_squared_error(y_test["time"][valid_test], med_test[valid_test])
    print(f"RMSE (Test): {rmse_test:.4f}")
else:
    print("⚠️ No valid median predictions (Test)")

# --- Train set ---
c_index_train = concordance_index_censored(y_train["event"], y_train["time"], coxph.predict(X_train))[0]
print(f"\nC-index (Train): {c_index_train:.4f}")

med_train = predict_median_survival(coxph, X_train, y_train)
valid_train = ~np.isnan(med_train)
if valid_train.sum() > 0:
    rmse_train = root_mean_squared_error(y_train["time"][valid_train], med_train[valid_train])
    print(f"RMSE (Train): {rmse_train:.4f}")
else:
    print("⚠️ No valid median predictions (Train)")


In [ ]:


# Get a few test patients
X_test_sel = X_test[:5]
y_test_sel = y_test[:5]
risk_scores = coxph.predict(X_test_sel)

# --- A. Estimated Survival Curves ---
plt.figure(figsize=(8, 5))
for i, (x, score) in enumerate(zip(X_test_sel, risk_scores)):
    km_times, km_surv = kaplan_meier_estimator(y_train["event"], y_train["time"])
    s_func = StepFunction(km_times, np.power(km_surv, np.exp(score)))
    plt.step(km_times, s_func(km_times), where="post", label=f"Patient {i+1}")

plt.title("Estimated Survival Curves (CoxPH)")
plt.xlabel("Time (Days)")
plt.ylabel("Probability of Survival")
plt.grid(True)
plt.legend()
plt.show()

# --- B. Cumulative Hazard Curves (from survival) ---
plt.figure(figsize=(8, 5))
for i, (x, score) in enumerate(zip(X_test_sel, risk_scores)):
    cumhaz = -np.log(np.clip(np.power(km_surv, np.exp(score)), 1e-6, 1.0))
    plt.step(km_times, cumhaz, where="post", label=f"Patient {i+1}")

plt.title("Estimated Cumulative Hazard (CoxPH)")
plt.xlabel("Time (Days)")
plt.ylabel("Cumulative Risk of Recurrence")
plt.grid(True)
plt.legend()
plt.show()
